Given a dataset where in name field, we are seeing unwanted data (Di,.byanshu, S..,hiv, like this..) using SQL or Pyspark, how can we clean this ? this cleaning should be done for 100k of recordset

In [0]:
from pyspark.sql.functions import *;
data = [("Di,.byanshu",), ("S..,hiv",), ("A..nkit",)]
df = spark.createDataFrame(data, ["name"])

cleaned_df = df.withColumn(
    "clean_name",
    trim(regexp_replace("name", "[^a-zA-Z ]", ""))  # remove everything except alphabets/spaces
)

cleaned_df.show(truncate=False)


In [0]:
%sql
-- Create table
CREATE TABLE data_engineering_practice.sql.names_table (
    name STRING
);

-- Insert messy names
INSERT INTO data_engineering_practice.sql.names_table (name) VALUES
('Di,.byanshu'),
('S..,hiv'),
('An,,kit'),
('Pri.,ya'),
('Ro..hit'),
('Sh,.,reya'),
('Su.,,raj'),
('A..nanya');


In [0]:
%sql
SELECT 
    name, TRIM(REGEXP_REPLACE(name, '[^a-zA-Z]', '')) AS clean_name
FROM data_engineering_practice.sql.names_table;


### 1. Find duplicate rows in a table

Question: Identify rows that occur more than once in the customers table (based on all columns).

In [0]:
from pyspark.sql import functions as F

df.groupBy("name", "email") \
  .count() \
  .filter("count > 1") \
  .show()


In [0]:
%sql
SELECT name, email, COUNT(*) AS cnt
FROM customers
GROUP BY name, email
HAVING COUNT(*) > 1;


### 2. Delete duplicate rows but keep one (deduplicate)

Question: Remove duplicates from transactions, keeping only one row per txn_id.

In [0]:
WITH cte AS (
  SELECT *, ROW_NUMBER() OVER(PARTITION BY txn_id ORDER BY txn_time DESC) AS rn
  FROM transactions
)
DELETE FROM transactions WHERE rn > 1;


In [0]:
from pyspark.sql.window import Window

w = Window.partitionBy("txn_id").orderBy(F.col("txn_time").desc())

df = df.withColumn("rn", F.row_number().over(w)) \
       .filter("rn = 1") \
       .drop("rn")


### 3. Find first occurrence and remove later duplicates

Question: From employees, keep only the first record of each employee based on hire_date.

In [0]:
%sql
SELECT *
FROM (
   SELECT *, ROW_NUMBER() OVER(PARTITION BY emp_id ORDER BY hire_date ASC) AS rn
   FROM employees
) t
WHERE rn = 1;


In [0]:
w = Window.partitionBy("emp_id").orderBy("hire_date")

df = df.withColumn("rn", F.row_number().over(w)) \
       .filter("rn = 1") \
       .drop("rn")


### 4. Count how many duplicates each record has

Question: For orders, show each order_id with a count of how many duplicates it has.

In [0]:
%sql
SELECT order_id, COUNT(*) AS dup_count
FROM orders
GROUP BY order_id
HAVING COUNT(*) > 1;


In [0]:
df.groupBy("order_id") \
  .count() \
  .filter("count > 1") \
  .withColumnRenamed("count", "dup_count") \
  .show()


### 5. Deduplicate based on multiple columns

Question: In sales, remove duplicates where both customer_id and product_id repeat, but keep the latest sale_date.

In [0]:
%sql
SELECT *
FROM (
   SELECT *, ROW_NUMBER() OVER(PARTITION BY customer_id, product_id ORDER BY sale_date DESC) AS rn
   FROM sales
) t
WHERE rn = 1;


In [0]:
w = Window.partitionBy("customer_id", "product_id").orderBy(F.col("sale_date").desc())

df = df.withColumn("rn", F.row_number().over(w)) \
       .filter("rn = 1") \
       .drop("rn")
